In this notebook I'll reconstruct the process and results of the DTCR algorithm presented in the "Learning Representations for Time Series Clustering".

In [1]:
# Imports
import torch
from torch.utils.data import DataLoader
from Utilities.DTCR import DTCRModel, DTCRConfig
from Utilities.UCRParser import read_dataset
from Utilities.DRNN import BidirectionalDRNN

In [2]:
train_ds, test_ds = read_dataset("Two_Patterns")
bs = 100
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=bs, shuffle=True)

config = DTCRConfig()
config.class_num = train_ds.number_of_labels
config.input_size = train_ds[0][0].shape[1]
config.num_steps = train_ds[0][0].shape[0]
config.batch_size = bs


Loading the Two_Patterns dataset...
The dataset Two_Patterns was loaded.


In [3]:
dtcr_model = DTCRModel(config)
dtcr_model

DTCRModel(
  (encoder): BidirectionalDRNN(
    (_regular_drnn): DRNN(
      (_layers): Sequential(
        (0): GRU(1, 100, batch_first=True)
        (1): GRU(100, 50, batch_first=True)
        (2): GRU(50, 50, batch_first=True)
      )
    )
    (_backwards_drnn): DRNN(
      (_layers): Sequential(
        (0): GRU(1, 100, batch_first=True)
        (1): GRU(100, 50, batch_first=True)
        (2): GRU(50, 50, batch_first=True)
      )
    )
  )
  (decoder): DTCRDecoder(
    (_rnn): GRU(400, 400, batch_first=True)
    (_linear): Linear(in_features=400, out_features=1, bias=True)
  )
  (classifier): Sequential(
    (0): Linear(in_features=400, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=2, bias=True)
    (3): Softmax(dim=1)
  )
)

In [4]:

recons_criterion = config.decoding_criterion()
classify_criterion = config.classifier_criterion()
optimizer = config.optimizer(dtcr_model.parameters(),
                             eps=config.learning_rate)
for epoch in range(1000):
    print("Starting epoch {}:".format(epoch + 1))
    dtcr_model.train_step(train_dl, test_dl, recons_criterion, classify_criterion,
                          optimizer)

Starting epoch 1:
[5] loss: 1.6857592344284058, classify: 0.6935059189796448, recons: 0.9922533392906189
[10] loss: 1.6856550931930543, classify: 0.6934138894081116, recons: 0.9922412037849426
Starting epoch 2:
[5] loss: 1.6855370283126831, classify: 0.6933412909507751, recons: 0.9921957135200501
[10] loss: 1.6855499982833861, classify: 0.6932860255241394, recons: 0.9922639608383179
Starting epoch 3:
[5] loss: 1.68546781539917, classify: 0.6932450771331787, recons: 0.9922227382659912
[10] loss: 1.6854221820831299, classify: 0.6932159304618836, recons: 0.9922062516212463
Starting epoch 4:
[5] loss: 1.6853947162628173, classify: 0.6931939840316772, recons: 0.9922006964683533
[10] loss: 1.6853745222091674, classify: 0.6931786298751831, recons: 0.9921958923339844
Starting epoch 5:
[5] loss: 1.6853464603424073, classify: 0.6931681632995605, recons: 0.9921782851219177
[10] loss: 1.685350728034973, classify: 0.6931610345840454, recons: 0.9921896815299988
Starting epoch 6:
[5] loss: 1.68533799

KeyboardInterrupt: 

In [ ]:
for name, param in dtcr_model.named_parameters():
    print(name, param.grad.norm())

encoder._regular_drnn._layers.0.weight_ih_l0 tensor(0.0151)
encoder._regular_drnn._layers.0.weight_hh_l0 tensor(0.0045)
encoder._regular_drnn._layers.0.bias_ih_l0 tensor(0.0073)
encoder._regular_drnn._layers.0.bias_hh_l0 tensor(0.0036)
encoder._regular_drnn._layers.1.weight_ih_l0 tensor(0.0051)
encoder._regular_drnn._layers.1.weight_hh_l0 tensor(0.0017)
encoder._regular_drnn._layers.1.bias_ih_l0 tensor(0.0046)
encoder._regular_drnn._layers.1.bias_hh_l0 tensor(0.0023)
encoder._regular_drnn._layers.2.weight_ih_l0 tensor(0.0021)
encoder._regular_drnn._layers.2.weight_hh_l0 tensor(0.0009)
encoder._regular_drnn._layers.2.bias_ih_l0 tensor(0.0028)
encoder._regular_drnn._layers.2.bias_hh_l0 tensor(0.0014)
encoder._backwards_drnn._layers.0.weight_ih_l0 tensor(0.0109)
encoder._backwards_drnn._layers.0.weight_hh_l0 tensor(0.0029)
encoder._backwards_drnn._layers.0.bias_ih_l0 tensor(0.0052)
encoder._backwards_drnn._layers.0.bias_hh_l0 tensor(0.0026)
encoder._backwards_drnn._layers.1.weight_ih_l0 t